In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
import os
import tempfile
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
ratings=tfds.load('movielens/100k-ratings',split='train')
movies=tfds.load('movielens/100k-movies',split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteJFBCT9/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [ ]:
ratings_df = tfds.as_dataframe(ratings)
ratings_df.head(10)

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4,b'doctor',4.0,b'53211'
1,25.0,"[4, 14]",b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5,b'entertainment',2.0,b'80525'
2,18.0,[4],b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17,b'student',4.0,b'55439'
3,50.0,"[5, 7]",b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4,b'healthcare',4.0,b'06472'
4,50.0,"[10, 16]",b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18,b'technician',3.0,b'75094'
5,18.0,"[7, 16]",b'325',b'Crash (1996)',19.0,876346551,False,b'601',1,b'artist',4.0,b'99687'
6,18.0,"[2, 3, 4, 12]",b'95',b'Aladdin (1992)',19.0,882064434,True,b'710',17,b'student',3.0,b'92020'
7,25.0,"[0, 5, 14]",b'92',b'True Romance (1993)',34.0,875135363,True,b'833',21,b'writer',2.0,b'90019'
8,25.0,[4],b'425',b'Bob Roberts (1992)',27.0,880844102,True,b'916',18,b'engineer',5.0,b'N2L5N'
9,25.0,"[0, 1, 15, 18]",b'271',b'Starship Troopers (1997)',32.0,884801053,True,b'940',2,b'administrator',2.0,b'02215'


In [ ]:
ratings = ratings.map(lambda x: {
    "user_gender": x["user_gender"],
    "movie_genres": x["movie_genres"],
    "bucketized_user_age": x["bucketized_user_age"],
})
movies = movies.map(lambda x: x["movie_title"])

In [ ]:
unique_movie_titles=np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_gender=np.unique(list(ratings.map(lambda x: x["user_gender"])))
unique_bucketized_user_age=np.unique(list(ratings.map(lambda x: x["bucketized_user_age"])))
unique_user_occupation_label=np.unique(list(ratings.map(lambda x: x["user_occupation_label"])))

In [ ]:
class UserModel(tf.keras.Model):
   
  def __init__(self):
    self.age_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_bucketized_user_age, mask_token=None),
        tf.keras.layers.Embedding(len(unique_bucketized_user_age) + 1, 32),
    ])
 
    self.gender_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_gender, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_gender) + 1, 32),
    ])

    self.mlb = MultiLabelBinarizer()
    self.t=self.mlb.fit(ratings_df['movie_genres'])
    self.movie_genre=tf.keras.Sequential([
        tf.keras.layers.Embedding(len(self.t) + 1, 32),
    ])

  def call(self, inputs):
    t_transformed=self.mlb.transform(inputs["movie_genres"])
    return tf.concat([
        self.age_embedding(inputs["bucketized_user_age"]),
        self.gender_embedding(inputs["user_gender"]),
        self.movie_genre(t_transformed),
    ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
 
  def __init__(self, layer_sizes):
    self.embedding_model = UserModel()
    self.dense_layers = tf.keras.Sequential()
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))
    self.dense_layers.add(tf.keras.layers.Dense(layer_sizes[-1]))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class MovieModel(tf.keras.Model):
   
  def __init__(self):
    super().__init__()
    max_tokens = 10_000
 
    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])
    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)
 
    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
    ])
 
    self.title_vectorizer.adapt(movies)
 
  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)